In [52]:
import pandas as pd
import numpy as np
import json
import folium

# Foursquare

In [266]:
# Export the foursquare dataset
foursquare = pd.read_csv('../data/foursquare/all.csv')

# Drop duplicates
foursquare.drop_duplicates(subset = 'id', inplace=True, ignore_index=True)

# Process the data to group categories
def process_categories(category_name):
    
    # Restaurant and Bar
    restaurant = ['restaurant', 'bar', 'café', 'cafe', 'coffee', 'pizza', 'sandwich', 'breakfast spot', 'ice cream',
                  'frozen yogurt', 'speakeasy', 'nightlife', 'diner', 'lounge', 'joint', 'snack', 'bistro', 'food truck',
                  'brewery', 'strip club', 'pub', 'nightclub', 'food court', 'steakhouse', 'tea room', 'taco', 'castle',
                  'creperie', 'buffet', 'food service', 'bath house', 'poke place', 'soup place', 'club house',
                  'fish & chips', 'burrito', 'poutine', 'noodle', 'brasserie', 'well', 'salad place', 'beer garden']
    for r in restaurant:
        if r in str(category_name).lower() and str(category_name) not in ['Salon / Barbershop', 'Piano Bar', 'Public Art']:
            return 'Restaurant/Bar'
    
    # Kids care (school, day care, playground, ...)
    kids = ['elementary school', 'middle school', 'preschool', 'care', 'playground']
    for k in kids:
        if k in str(category_name).lower() or str(category_name) == 'School':
            return 'KidCare'
    
    # Education
    education = ['school', 'college & university', 'university', 'adult education center']
    for e in education:
        if e in str(category_name).lower() and str(category_name) not in ['Driving School']:
            return 'EducationGraduate'
    
    # Entertainment
    entertainment = ['museum', 'theater', 'entertainment', 'event space', 'art gallery', 'performing', 'public art',
                     'art studio', 'library', 'design', 'music venue', 'comedy', 'photography', 'sculpture',
                     'convention center', 'arcade', 'bowling', 'recreation', 'rock', 'jazz', 'concert', 'auditorium',
                     'water', 'monument', 'lighthouse', 'exhibit', 'planetarium', 'recording studio', 'memorial site',
                     'piano', 'circus', 'zoo', 'spa', 'massage', 'casino', 'play area']
    for e in entertainment:
        if e in str(category_name).lower():
            return 'Entertainment/Culture'
    
    # Services
    services = ['store', 'shop', 'service', 'pharmacy', 'bank', 'bakery', 'gas station', 'government', 'bodega',
                'salon', 'post', 'rental car', 'veterinarian', 'boutique', 'car wash', 'tattoo', 'butcher', 'real estate',
                'auto', 'storage', 'market', 'parking', 'atm', 'lawyer', 'funeral', 'food', 'cleaner', 'embassy',
                'driving', 'agency', 'courthouse', 'repair', 'winery', 'grocery', 'animal', 'notary', 'food stand',
                'credit union', 'locksmith', 'laundromat', 'general travel']
    for s in services:
        if s in str(category_name).lower():
            return 'Services'
    
    # Sports
    sports = ['pool', 'gym', 'sports', 'field', 'studio', 'court', 'rink', 'golf', 'arena', 'track', 'stadium',
              'skate', 'range', 'dojo', 'curling']
    for s in sports:
        if s in str(category_name).lower():
            return 'Sports'
    
    # Outdoor
    outdoor = ['park', 'outdoor', 'scenic lookout', 'site', 'garden', 'marina', 'boat', 'plaza', 'trail', 'mountain', 'surf',
               'campground', 'lake', 'beach', 'rest area', 'vineyard', 'forest', 'nature', 'cave', 'fishing', 'rafting']
    for o in outdoor:
        if o in str(category_name).lower():
            return 'Outdoor'
    
    # Medical
    medical = ['dentist', 'doctor', 'medical', 'hospital', 'assisted living', 'chiropractor', 'mental', 'healer',
               'emergency', 'therapist', 'laboratory', 'acupuncturist', 'clinic']
    for m in medical:
        if m in str(category_name).lower():
            return 'Medical'
    
    # Religious
    religious = ['church', 'synagogue', 'temple', 'spiritual', 'mosque', 'shrine']
    for r in religious:
        if r in str(category_name).lower():
            return 'Religious'
    
    # All other
    if pd.isna(category_name):
        category = 'Other'

    return 'Other'
    

foursquare['category'] = foursquare['category_name'].apply(process_categories)
foursquare['category'].value_counts()

Services                 3400
Other                    3105
Restaurant/Bar           1958
Entertainment/Culture     717
Medical                   456
Sports                    451
Outdoor                   416
KidCare                   274
Religious                 172
EducationGraduate         153
Name: category, dtype: int64

# Map

In [2]:
census = pd.read_csv('../data/census/all.csv')
test = census[census['Category'] == '  Revenu total médian des ménages en 2015 ($)'][['FSA', 'CategoryValue']]
test

,FSA,CategoryValue
58,H1A,62608.0
218,H1B,53437.0
378,H1C,86949.0
538,H1E,61012.0
698,H1G,42814.0
...,...,...
15578,H9P,72229.0
15738,H9R,80064.0
15898,H9S,67785.0
16058,H9W,123392.0


In [3]:
latitude = 45.505331312
longitude = -73.55249779
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=10)
quebec_geo = "../data/mtl_fsa_map.geojson"
map_montreal.choropleth(geo_data=quebec_geo,
    data = test,
    columns=['FSA','CategoryValue'],
    key_on='feature.properties.CFSAUID',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Median revenue by FSA')   
    
map_montreal

c:\users\noemi\appdata\local\programs\python\python38\lib\site-packages\folium\folium.py:411: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [4]:
map_mtl = folium.Map(location=[latitude, longitude], zoom_start=10)

map_mtl.choropleth(geo_data = quebec_geo, fill_opacity = 0)
# add markers to map
for lat, lng in zip(centris['Latitude'], centris['Longitude']):
#     label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
#         popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mtl)  
    
map_mtl

NameError: name 'centris' is not defined